# 1. Initialize the environment

In [ ]:
!pip install --upgrade pip
!pip install langchain openai pandas tabulate

In [1]:
from langchain.agents import create_csv_agent
from langchain.llms import OpenAI
from langchain.agents.agent_types import AgentType
import pandas as pd

Downloading CSV from the [DOU Salaries repo](https://github.com/devua/csv/tree/master/salaries)

In [3]:
!curl -OL https://github.com/devua/csv/raw/master/salaries/2023_june_raw.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0
100 6214k  100 6214k    0     0  1667k      0  0:00:03  0:00:03 --:--:-- 3296k


In [2]:
OPEN_AI_KEY = "YourOpenAIAPIKey"

In [21]:
agent = create_csv_agent(
    OpenAI(
        temperature=0,
        openai_api_key=OPEN_AI_KEY,
    ),
    "2023_june_raw.csv",
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    pandas_kwargs={"delimiter": ";"},
)

# 2. Agent usage

In [22]:
agent.run(
    "What is the average salary of a software engineer in Ukraine?"
)



> Entering new AgentExecutor chain...
Thought: I need to find the salaries of software engineers in Ukraine
Action: python_repl_ast
Action Input: df[(df['Спеціалізація'] == 'Software Engineer') & (df['Де ви зараз живете?'] == 'В Україні')]['Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків'].mean()
Observation: TypeError: can only concatenate str (not "int") to str
Thought: I need to convert the salary column to a string
Action: python_repl_ast
Action Input: df[(df['Спеціалізація'] == 'Software Engineer') & (df['Де ви зараз живете?'] == 'В Україні')]['Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків'].astype(str).mean()
Observation: TypeError: Could not convert ... to numeric
Thought:

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 13146 tokens (12890 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

As we can see, the agent failed to answer the question.
It was quite close to the answer, but it failed to understand the error.

# 3. Resolve the task

Let's load data and run the same operations as the agent did.

In [49]:
df = pd.read_csv("2023_june_raw.csv", delimiter=";")

In [50]:
df[(df['Спеціалізація'] == 'Software Engineer') & (df['Де ви зараз живете?'] == 'В Україні')]['Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків'].mean()

TypeError: can only concatenate str (not "int") to str

Agent saw this error too, but he handled it incorrectly.

Let's filter out the extra data.

In [51]:
df = df[(df['Спеціалізація'] == 'Software Engineer') & (df['Де ви зараз живете?'] == 'В Україні')]

In [52]:
df

,Timestamp,Ваша основна зайнятість в ІТ зараз...,"Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків",Спеціалізація,Ваш тайтл на цій посаді,Вкажіть вашу спеціалізацію Game Design,Вкажіть вашу спеціалізацію Sound,Вкажіть вашу спеціалізацію - QA,Оберіть вашу посаду Designer / Artist,"Якщо ви працюєте в GameDev, оберіть вашу спеціалізацію",Ваша посада Marketing,Оберіть вашу посаду HR,Ваша посада Management,Ваша посада Analyst,Ваша посада Data Science,"Ваша посада DevOps, SRE","Ваша посада CTO, Director",Ваша спеціалізація Support,Чи використовуєте ви у своїй роботі мови програмування (одну чи декілька)?,Основна мова програмування,Вкажіть вашу основну спеціалізацію,"В якій сфері проєкт, в якому ви зараз працюєте?","Основний напрям роботи компанії, в якій працюєте",Кількість спеціалістів у вашій компанії (в Україні/з України),Загальний стаж роботи за нинішньою ІТ-спеціальністю,Яка у вас освіта?,Знання англійської мови,Де ви зараз живете?,В якій області ви зараз живете?,Ваша стать,Ваш вік
1,5.26.2023 13:40:36,Працюю full-time в ІТ-компанії чи ІТ-відділі,3450,Software Engineer,Middle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",Dart,Mobile розробка,Medtech / Healthcare,Аутсорсингова,понад 1000,4,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Pre-Intermediate,В Україні,Київ чи область,Чоловік,28
3,5.26.2023 13:41:19,Працюю full-time в ІТ-компанії чи ІТ-відділі,4000,Software Engineer,Middle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",JavaScript,Front-end розробка,E-commerce,Аутстафінгова,понад 1000,5,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Upper-Intermediate,В Україні,Суми чи область,Чоловік,27
7,5.26.2023 13:42:24,Працюю full-time в ІТ-компанії чи ІТ-відділі,4170,Software Engineer,Senior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",C++,Back-end розробка,Telecom,Аутстафінгова,до 200,4,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Upper-Intermediate,В Україні,Львів чи область,Чоловік,28
8,5.26.2023 13:42:32,Працюю full-time в ІТ-компанії чи ІТ-відділі,1150,Software Engineer,Junior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",PHP,Back-end розробка,"E-commerce, Logistics",Аутсорсингова,до 50,2,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Upper-Intermediate,В Україні,Чернівці чи область,Чоловік,35
9,5.26.2023 13:42:42,Працюю full-time в ІТ-компанії чи ІТ-відділі,630,Software Engineer,Middle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Front-end розробка,"E-commerce, Real estate",Продуктова,до 10 спеціалістів,2,Незакінчена вища,Pre-Intermediate,В Україні,Черкаси чи область,Чоловік,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14041,6.26.2023 12:09:34,Працюю full-time в ІТ-компанії чи ІТ-відділі,3550,Software Engineer,Senior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JavaScript,Full Stack розробка,ERP,Аутсорсингова,до 50,6,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Intermediate,В Україні,Вінниця чи область,Чоловік,31
14046,6.26.2023 12:35:26,Працюю full-time в ІТ-компанії чи ІТ-відділі,500,Software Engineer,Junior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",JavaScript,Front-end розробка,E-commerce,Фріланс,до 200,1,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Intermediate,В Україні,Київ чи область,Чоловік,24
14048,6.26.2023 12:49:30,Працюю full-time в ІТ-компанії чи ІТ-відділі,1000,Software Engineer,Junior,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",C# / .NET,Full Stack розробка,Gambling,Аутстафінгова,до 200,1,"Вища (бакалавр, спеціаліст, магістр) - одна аб...",Intermediate,В Україні,Львів чи область,Чоловік,30
14051,6.26.2023 13:10:47,Працюю full-time в ІТ-компанії чи ІТ-відділі,3250,Software Engineer,Middle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Так, використовую",JavaScript,Front-end розробк

In [53]:
salary_column_name = 'Зарплата / дохід у $$$ за місяць, лише ставка після сплати податків'

The problem was connected with the comma in the salary column.
Let's replace it with a dot. And convert the column to float.

In [60]:
df[salary_column_name] = df[salary_column_name].apply(lambda x: str(x).replace(',', '.'))

In [61]:
df[salary_column_name] = df[salary_column_name].astype(float)

Now we can calculate the mean salary — the answer to the question.

In [62]:
df[salary_column_name].mean()

3556.528067841703

As we can see, the LangChain agent was right, this command returns the correct answer. However, it was not able to understand the error in the data, and it was not able to fix it.